# PEC model

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Import your melt inclusion and olivine data. 

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

If you have data for internal inclusion pressures (in bars) you can also import those

In [14]:
pressure_file ="./data/pressure.csv"

pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

pressure

name
PI032-04-01    4104.410589
PI032-04-02    4231.966408
PI041-02-02    1956.244091
PI041-03-01    3548.843351
PI041-03-03    3500.514426
PI041-05-04    1386.274577
PI041-05-06    2512.013195
PI041-07-01    3668.361134
PI041-07-02    2449.116051
PI052-01-02    1404.218415
Name: P_bar, dtype: float64

Here we will also use a whole-rock dataset to set up a model for predicting the initial FeO contents of our melt inclusions. For explanation of this model, please follow the [Initial FeO prediction](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) example.

In [4]:
wholerock_file = "./data/wholerock.csv"

wholerock = mp.read_melt(wholerock_file, index_col=["name"])

Let's begin by setting up the model to predict the initial FeO content of our melt inclusions according to the previous [example](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#). Here, the FeO prediction model is based on TiO<sub>2</sub>, Al<sub>2</sub>O<sub>3</sub> and CaO, which we can check by accessing their coefficients:

In [5]:
x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

In [6]:
FeOi_predict.coefficients

intercept    11.585125
TiO2          1.463365
Al2O3        -0.230722
CaO          -0.169262
dtype: float64

To use this model during PEC correction, we need to store this model as a callable function, which we can do with the *model* attribute:

In [7]:
FeO_model = FeOi_predict.model

Let's first quickly test the model by predicting FeO contents for our uncorrected melt compositions:

In [8]:
FeO_model(melt)

name
PI032-04-01    10.196984
PI032-04-02    10.265411
PI041-02-02    10.221895
PI041-03-01    10.614069
PI041-03-03    10.716794
PI041-05-04     9.420381
PI041-05-06    11.476340
PI041-07-01    11.608516
PI041-07-02    11.451018
PI052-01-02     8.722012
dtype: float32

This all looks good, so let continue with setting up the PEC correction model.

First, preview the melt and olivine data to check everything looks ok:

In [9]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,50.028862,15.226979,5.330006,10.522019,8.59454,3.914280,0.730798,0.126266,2.669216,0.303542,NaN,0.658252,1.508379,0.082590,0.152337,0.035565
PI032-04-02,49.558861,16.041588,5.117479,10.266067,8.25471,3.782408,0.997826,0.129969,2.814807,0.356900,NaN,0.713472,1.412274,0.089010,0.175511,0.047265
PI041-02-02,49.112045,16.972038,4.864153,9.195718,10.07540,3.793512,1.079184,0.153456,2.807967,0.561756,NaN,0.464520,0.656341,0.047715,0.068108,0.021185
PI041-03-01,47.098515,17.448515,4.650640,12.154110,7.95402,3.707263,1.268282,0.101370,3.493271,0.611504,NaN,0.883720,0.324968,0.088070,0.096228,0.059958
PI041-03-03,46.478870,17.637102,4.762752,12.364033,7.84027,3.789446,1.294603,0.077221,3.617484,0.574248,NaN,0.910470,0.344988,0.090605,0.088145,0.061584


In [10]:
olivine.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO,total
name,,,,,,,,
PI032-04-01,38.204800,15.981400,44.194000,0.172665,0.234256,0.005229,0.240552,99.032906
PI032-04-02,38.638500,15.898400,43.467400,0.188024,0.219599,-0.007310,0.234622,98.639240
PI041-02-02,37.270100,20.815901,41.015099,0.104744,0.293681,0.016547,0.214939,99.731000
PI041-03-01,38.795799,15.469300,44.750999,0.180198,0.212197,0.031333,0.259919,99.699745
PI041-03-03,38.701900,15.782500,44.920799,0.164372,0.214836,0.037263,0.275049,100.096720


Make sure that each row in *melt* and *olivine* is a matching pair of melt inclusion and olivine host. Here we use the sample names stored in the indices of both dataframes:

In [11]:
print(melt.index, "\n\n", olivine.index)

Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name') 

 Index(['PI032-04-01', 'PI032-04-02', 'PI041-02-02', 'PI041-03-01',
       'PI041-03-03', 'PI041-05-04', 'PI041-05-06', 'PI041-07-01',
       'PI041-07-02', 'PI052-01-02'],
      dtype='object', name='name')


We can verify that the indices are the same with the *equals* method:

In [12]:
melt.index.equals(olivine.index)

True

Check the configuration of MagmaPandas and the PEC model and make changes if you want to (see the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html)). You can change MagmaPandas settings either via the MagmaPandas object (here: mp.configuration) or the MagmaPEC object (here: mpc.configuration).

In [13]:
print(mpc.configuration)
print(mpc.PEC_configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
Δ buffer....................................1
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed

############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %)..........................0.05    
Kd convergence...................................0.001   


Now we can initialise the PEC model, where we need the following data:

- inclusions

    inclusion major element compositions in oxide wt. % as a MagmaPandas Melt frame.

- olivines

    olivine major element compositions in oxide wt. % as a MagmaPandas Olivine frame.

- P_bar

    pressures in bar at which to run the model. You can use a fixed pressure for all inclusions, e.g. *P_bar=2e3* for 2 kbar, or indicate specific pressures per inclusion. In this example we do the latter, by passing the above imported pandas Series with internal inclusion pressures

- FeO_target

    Estimated initial FeO contents of melt inclusions. You can use a fixed value for all melt inclusions, e.g. FeO = 11, specific values for individual melt inclusions, stored in a pandas Series, or a predictive equation based on melt major element composition. This equation needs to be a callable function that accepts a Pandas DataFrame with melt compositions in oxide wt. % as input and return an array-like object with initial FeO contents per inclusion. In the example above we showed how to set up a function like this using MagmaPEC.

In [18]:
pec_model = mpc.PEC_olivine(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeO_model)

Now we simply run the model with the *correct* method. It runs in two stages: the first stage equilibrates Fe and Mg between melt inclusions and olivine hosts through Fe-Mg cation exchange and the second stage corrects melt inclusions back to their initial FeO contents by melting or crystallising olivine.

This method returns three objects:

- corrected melt compositions as a MagmaPandas Melt frame

- PEC extents as a pandas DataFrame
    
- inclusion entrapment temperatures as a pandas Series

In [19]:
melts_corrected, pec, temperatures = pec_model.correct()

Equilibrating ... |██████████████████████████████| 100% [10/10] in 1.6s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 4.4s 


The dataframe with pec extents has three columns:

- equilibration_crystallisation: 
    
    crystallisation extents during the equilibration stage

- PE_crystallisation:

    crystallisation extents during the crystallisation stage

- total_crystallisation:

    total amount of post-entrapment crystallisation

with all data in mol fractions

In [21]:
pec

,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-0.009964,0.084,0.074036
PI032-04-02,-0.007586,0.094,0.086414
PI041-02-02,0.002419,0.000,0.002419
PI041-03-01,-0.012079,0.132,0.119921
PI041-03-03,-0.002573,0.120,0.117427
PI041-05-04,-0.014385,-0.024,-0.038385
PI041-05-06,-0.008686,0.030,0.021314
PI041-07-01,-0.014803,0.124,0.109197
PI041-07-02,-0.010754,0.116,0.105246


Results are also stored in the pec object and can be accessed via the *inclusions* and *olivine_corrected* attributes, while entrapment temperatures can be calculated on the fly with the *temperature* method of the *inclusions* Melt frame (see the [MagmaPandas documentation](https://magmapandas.readthedocs.io/en/latest/code_documentation.html#MagmaPandas.MagmaFrames.Melt.temperature))

In [26]:
pec_model.inclusions

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.142871,14.286756,6.846292,9.887887,10.268466,3.672495,0.685656,0.133863,2.504339,0.284792,0.000000,0.617592,1.415207,0.077488,0.142927,0.033368,100.0
PI032-04-02,48.643162,14.909829,6.767915,9.559174,10.326331,3.515678,0.927461,0.136766,2.616311,0.331732,0.000000,0.663159,1.312682,0.082733,0.163135,0.043932,100.0
PI041-02-02,49.124917,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.000000,0.463920,0.655494,0.047654,0.068020,0.021157,100.0
PI041-03-01,45.977682,15.804043,6.850408,11.031140,10.686113,3.357234,1.148535,0.111869,3.163447,0.553768,0.000000,0.800282,0.294285,0.079754,0.087142,0.054297,100.0
PI041-03-03,45.366859,15.986441,6.743449,11.228231,10.825106,3.434104,1.173206,0.088408,3.278267,0.520400,0.000000,0.825094,0.312638,0.082108,0.079880,0.055809,100.0
PI041-05-04,47.820789,18.687631,3.754351,9.446385,9.338353,4.624027,1.615288,0.136903,2.500030,0.829647,0.000000,0.512805,0.464129,0.088230,0.122093,0.059338,100.0
PI041-05-06,46.412729,17.038781,4.687957,8.958150,11.492975,4.017958,1.422544,0.167353,3.648689,0.621988,0.000000,0.642653,0.588106,0.107850,0.128636,0.063632,100.0
PI041-07-01,45.843441,15.436316,6.763475,9.807944,11.592497,3.172540,1.289405,0.145093,3.579769,0.562287,0.000000,0.473979,1.034465,0.076104,0.165607,0.057078,100.0
PI041-07-02,45.903942,15.711951,6.617583,10.161595,11.421153,3.218056,1.372311,0.134870,3.565996,0.621557,0.000000,0.375504,0.621727,0.074231,0.141648,0.057877,100.0


In [23]:
pec_model.olivine_corrected

,equilibration_crystallisation,PE_crystallisation,total_crystallisation
name,,,
PI032-04-01,-0.009964,0.084,0.074036
PI032-04-02,-0.007586,0.094,0.086414
PI041-02-02,0.002419,0.000,0.002419
PI041-03-01,-0.012079,0.132,0.119921
PI041-03-03,-0.002573,0.120,0.117427
PI041-05-04,-0.014385,-0.024,-0.038385
PI041-05-06,-0.008686,0.030,0.021314
PI041-07-01,-0.014803,0.124,0.109197
PI041-07-02,-0.010754,0.116,0.105246


In [24]:
pec_model.inclusions.temperature(P_bar=pressure)

name
PI032-04-01    1453.152003
PI032-04-02    1453.695703
PI041-02-02    1404.622590
PI041-03-01    1467.407078
PI041-03-03    1465.314443
PI041-05-04    1378.336232
PI041-05-06    1409.681657
PI041-07-01    1458.316437
PI041-07-02    1455.514279
PI052-01-02    1372.246336
Name: T_K, dtype: float64